In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


meta = pd.read_csv('/workspaces/stimage/data/meta_all_gene.csv')
path = '/workspaces/stimage/data/raw'

data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'lung'),:]

# load data
i = 1
slide = data['slide'][i]
gene_exp_slide = pd.read_csv(f'{path}/{data.tech[i]}/gene_exp/{data.slide[i]}_count.csv',sep=',',index_col=0)
adata = sc.AnnData(gene_exp_slide)
adata.var_names_make_unique()

sc.pp.filter_cells(adata, min_genes=1)
sc.experimental.pp.highly_variable_genes(adata, n_top_genes=128)

In [3]:
data = meta.loc[(meta['species'] == 'human') & (meta['tissue'] == 'lung'),:]

In [6]:
data

,slide,species,tissue,pmid,title,abstract,keywords,involve_cancer,tech,spot_num,gene_num
220,GSE178361_GSM5388414,human,lung,35355018,Human distal lung maps and lineage hierarchies...,Mapping the spatial distribution and molecular...,NaN,False,Visium,1175,36601
221,GSE178361_GSM5388415,human,lung,35355018,Human distal lung maps and lineage hierarchies...,Mapping the spatial distribution and molecular...,NaN,False,Visium,1045,36601
1101,Human_Lung_10X_02132023_Visium,human,lung,NaN,NaN,NaN,NaN,False,Visium,6195,18085
1102,Human_Lung_10X_07132022_Visium,human,lung,NaN,NaN,NaN,NaN,False,Visium,3858,18085
1146,Human_Lung_10X_03292024_VisiumHD,human,lung,NaN,NaN,NaN,NaN,False,VisiumHD,605471,18085


In [2]:
# sort genes by highly_variable
adata.var_names_make_unique()
hvg_list = adata.var['highly_variable_rank']
hvg_list = hvg_list.sort_values()
hvg_list = hvg_list.dropna()

adata_hvg = adata[:, hvg_list.index]

In [ ]:
array = np.array(adata_hvg.X)

In [ ]:
plt.figure(figsize=(8, 6))  # Optional: set figure size
plt.hist(array[:,1], bins=30, edgecolor='k', alpha=0.7)  # Plot histogram
plt.title(f'Distribution of Feature {i}')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)  # Optional: add grid
plt.show()


In [ ]:
array.shape

In [ ]:
# Create heatmap
sns.clustermap(array.T, cmap="viridis", figsize=(10, 10))
plt.show()


In [ ]:
sc.pp.normalize_total(adata_hvg)
sc.pp.log1p(adata_hvg)
hvg = adata_hvg.X
hvg = pd.DataFrame(hvg)
hvg.index = adata_hvg.obs.index

In [ ]:
sns.clustermap(np.array(hvg).T, cmap="viridis", figsize=(10, 10))
plt.show()